In [1]:
!pip install pandas openpyxl

In [3]:
from google.colab import files
uploaded = files.upload()

Saving karbon_ayak_izi.csv to karbon_ayak_izi.csv


In [4]:
import pandas as pd
df = pd.read_csv(next(iter(uploaded)))

In [5]:
df['ADRES1'] = df['ADRES1'].fillna('')
df['ADRES2'] = df['ADRES2'].fillna('')
df['SEHIR'] = df['SEHIR'].fillna('')

In [6]:
df['tam_adres'] = df['ADRES1'].astype(str) + ', ' + df['ADRES2'].astype(str) + ', ' + df['SEHIR'].astype(str) + ', Türkiye'

In [ ]:
import requests
import time

API_KEY = ""

def get_coordinates_google(address):
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY}"
        response = requests.get(url)
        data = response.json()
        if data['status'] == 'OK':
            loc = data['results'][0]['geometry']['location']
            return f"{loc['lat']}, {loc['lng']}"
        else:
            return "adres yok"
    except:
        return "adres yok"

df['koordinat'] = df['tam_adres'].apply(lambda x: get_coordinates_google(x))
time.sleep(0.1)

In [ ]:
df['FAF_ADRES'] = df['FAF_ADRES'].fillna('')
df['faf_adres_koordinat'] = df['FAF_ADRES'].apply(lambda x: get_coordinates_google(str(x)))
time.sleep(0.1)

In [ ]:
from geopy.distance import geodesic

def hesapla_mesafe(row):
    if row['koordinat'] != 'adres yok' and row['faf_adres_koordinat'] != 'adres yok':
        try:
            koord1 = tuple(map(float, row['koordinat'].split(',')))
            koord2 = tuple(map(float, row['faf_adres_koordinat'].split(',')))
            return round(geodesic(koord1, koord2).km, 2)
        except:
            return 'koordinat yok'
    else:
        return 'koordinat yok'

df['mesafe_km'] = df.apply(hesapla_mesafe, axis=1)

In [ ]:
import numpy as np

emisyon_faktor = 0.00025  # kg CO2e / kg.km
df['mesafe_km'] = pd.to_numeric(df['mesafe_km'], errors='coerce')

def hesapla_karbon(mesafe, agirlik):
    try:
        if pd.notna(mesafe) and isinstance(agirlik, (float, int)):
            return round(mesafe * agirlik * emisyon_faktor, 2)
        else:
            return "hesaplanamaz"
    except:
        return "hesaplanamaz"

df['karbon_kg'] = df.apply(lambda row: hesapla_karbon(row['mesafe_km'], row['TOPLAM_AGIRLIK']), axis=1)
df['mesafe_km'] = df['mesafe_km'].apply(lambda x: x if pd.notna(x) else "koordinat yok")

In [ ]:
df.to_excel("karbon_ayak_izi.xlsx", index=False)

In [ ]:
from google.colab import files
files.download("karbon_ayak_izi.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>